In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('/Users/ica/code/jp-diogo/business_sustainability/MergedData.csv')

In [12]:
# dam_zonal_lbmp is the price variable

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42815 entries, 0 to 42814
Data columns (total 30 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Unnamed: 0               42815 non-null  int64  
 1   timestamp                42815 non-null  object 
 2   zone_name                42815 non-null  object 
 3   dam_zonal_lbmp           42815 non-null  float64
 4   dam_zonal_losses         42815 non-null  float64
 5   dam_zonal_congestion     42815 non-null  float64
 6   dam_zonal_price_version  42815 non-null  int64  
 7   date                     42815 non-null  object 
 8   hour_of_day              42815 non-null  int64  
 9   rtd_actual_load          42815 non-null  float64
 10  dual_fuel                42815 non-null  float64
 11  dual_fuel_%              42815 non-null  float64
 12  hydro                    42815 non-null  float64
 13  hydro_%                  42815 non-null  float64
 14  natural_gas           

In [5]:
df.shape

(42815, 30)

In [ ]:
# Must previously build a df with the features we want to use and name it 'df'

In [ ]:
len_ = int(0.8*df.shape[0])
df_train = df[:len_]
df_test = df[len_:]

# Missing the 2+ days interval between train and test datasets

In [8]:
def subsample_sequence(df, length):
    """
    Given the initial dataframe `df`, return a shorter dataframe sequence of length `length`.
    This shorter sequence should be selected at random
    """
    # $CHALLENGIFY_BEGIN
    last_possible = df.shape[0] - length
    
    random_start = np.random.randint(0, last_possible)
    df_sample = df[random_start: random_start+length]
    # $CHALLENGIFY_END
    return df_sample

In [16]:
def split_subsample_sequence(df, length):
    '''Create one single random (X,y) pair'''
    # $CHALLENGIFY_BEGIN
    df_subsample = subsample_sequence(df, length)
    y_sample = df_subsample.iloc[length -1]['dam_zonal_lbmp']
    
    X_sample = df_subsample[0:length -1]
    X_sample = X_sample.values
    # $CHALLENGIFY_END
    return np.array(X_sample), np.array(y_sample)

In [17]:
#Not sure if the code below works

In [18]:
def split_subsample_sequence(df, length):
    '''Return one single random sample (X_sample, y_sample) containing one sequence each of length `length`'''

    df_subsample = subsample_sequence(df, length)
    
    y_sample = df_subsample.iloc[length-1]['dam_zonal_lbmp']
    # Case y_sample is NaN: redraw !
    if y_sample != y_sample: # A value is not equal to itself only for NaN
            X_sample, y_sample = split_subsample_sequence(df, length) # Recursive call !!!
            return np.array(X_sample), np.array(y_sample)

    X_sample = df_subsample[0:length-1]
        
    return np.array(X_sample), np.array(y_sample)

In [19]:
def get_X_y(df, n_sequences, length):
    '''Return a list of samples (X, y)'''
    # $CHALLENGIFY_BEGIN
    X, y = [], []

    for i in range(n_sequences):
        (xi, yi) = split_subsample_sequence(df, length)
        X.append(xi)
        y.append(yi)
        
    X = np.array(X)
    y = np.array(y)
    # $CHALLENGIFY_END
    return X, y

In [20]:
# Example of what the code would return 
X_train, y_train = get_X_y(df_train, 2000, 21)
X_test, y_test = get_X_y(df_test, 400, 21)